In [2]:
from setfit import AbsaModel

model = AbsaModel.from_pretrained(
    "tomaarsen/setfit-absa-paraphrase-mpnet-base-v2-restaurants-aspect",
    "tomaarsen/setfit-absa-paraphrase-mpnet-base-v2-restaurants-polarity"
)

/opt/homebrew/lib/python3.11/site-packages/torch/_utils.py:836: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/opt/homebrew/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_lg' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/opt/homebrew/lib/python3.11/site-packages/tensorflow_hub/__init__.py:61: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io

In [53]:
import csv
from googleapiclient.discovery import build

# Define your YouTube API key
api_key = os.environ.get("GOOGLE_API_KEY")

# Define the video ID of the YouTube video you want to fetch comments for
video_id = "dtp6b76pMak"

# Create a YouTube Data API client
youtube = build('youtube', 'v3', developerKey=api_key)

# Fetch comments from the YouTube video using the YouTube Data API
comments = []
next_page_token = None
count = 0

while count < 1000:
    response = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        maxResults=100,  # Adjust as needed
        pageToken=next_page_token
    ).execute()

    for item in response['items']:
        count += 1
        comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
        comments.append(comment)

    next_page_token = response.get('nextPageToken')

    if not next_page_token:
        break

# Write comments to a CSV file
with open("youtube_data.csv", "w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["comment_text"])  # Write header
    for comment in comments:
        writer.writerow([comment])


In [3]:
import pandas as pd
import swifter

# Read the CSV file
df = pd.read_csv("youtube_data.csv")

# Define a function to perform aspect-based sentiment analysis and store results in a new column
def perform_absa(review):
    absa_result = model.predict([review])[0]  # Get the first result
    return absa_result

# Apply the function to each row in the DataFrame using parallel processing
df['aspect_sentiment'] = df['comment_text'].swifter.apply(perform_absa)

# Save the DataFrame to a new CSV file
df.to_csv("output_file.csv", index=False)

Pandas Apply: 100%|██████████| 1000/1000 [04:02<00:00,  4.12it/s]


In [1]:
import pandas as pd
import ast

# Read the CSV file
df = pd.read_csv('output_file.csv')

# Drop rows where the aspect_sentiment column contains '[]'
df = df[df['aspect_sentiment'] != '[]']
df['aspect_sentiment'] = df['aspect_sentiment'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
# Display the resulting DataFrame
print(df['aspect_sentiment'])

1          [{'span': 'display', 'polarity': 'negative'}]
2      [{'span': 'helmet', 'polarity': 'positive'}, {...
7      [{'span': 'TVs', 'polarity': 'neutral'}, {'spa...
10     [{'span': 'killer apps', 'polarity': 'negative...
16     [{'span': 'orange site', 'polarity': 'positive...
                             ...                        
988      [{'span': 'AR headset', 'polarity': 'neutral'}]
990    [{'span': 'apps', 'polarity': 'neutral'}, {'sp...
991    [{'span': 'Lawnmower man', 'polarity': 'neutra...
995         [{'span': 'product', 'polarity': 'neutral'}]
997         [{'span': 'iphones', 'polarity': 'neutral'}]
Name: aspect_sentiment, Length: 474, dtype: object


In [20]:
import pandas as pd
import json
from collections import Counter

# Extract aspects and polarities and count occurrences
aspect_polarity_counts = Counter()

for entries in df['aspect_sentiment']:
    try: 
        for entry in entries:
            span = entry['span']
            polarity = entry['polarity']
            aspect_polarity_counts[(span, polarity)] += 1
    except:
        continue

# Separate counts for negative, positive, and neutral aspects
negative_aspects = {}
positive_aspects = {}
neutral_aspects = {}

for aspect_polarity, count in aspect_polarity_counts.items():
    aspect, polarity = aspect_polarity
    if polarity == 'negative':
        negative_aspects[aspect] = count
    elif polarity == 'positive':
        positive_aspects[aspect] = count
    else:
        neutral_aspects[aspect] = count

# Get the top 25 negative, positive, and neutral aspects
top_negative_aspects = sorted(negative_aspects.items(), key=lambda x: x[1], reverse=True)[:25]
top_positive_aspects = sorted(positive_aspects.items(), key=lambda x: x[1], reverse=True)[:25]
top_neutral_aspects = sorted(neutral_aspects.items(), key=lambda x: x[1], reverse=True)[:25]

# Store the top aspects in a dictionary
top_aspects = {
    "negative": top_negative_aspects,
    "positive": top_positive_aspects,
    "neutral": top_neutral_aspects
}

# Write the dictionary to a JSON file
with open('top_aspects.json', 'w') as f:
    json.dump(top_aspects, f, indent=4)

print("Top 25 aspects for each polarity are stored in 'top_aspects.json'.")


Top 25 aspects for each polarity are stored in 'top_aspects.json'.


In [21]:
import json
from setfit import AbsaModel

# Load the JSON file containing aspects and suggestions
with open('top_aspects.json', 'r') as f:
    aspects_with_suggestions = json.load(f)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
from setfit import AbsaModel

model = AbsaModel.from_pretrained(
    "tomaarsen/setfit-absa-paraphrase-mpnet-base-v2-restaurants-aspect",
    "tomaarsen/setfit-absa-paraphrase-mpnet-base-v2-restaurants-polarity"
)

/opt/homebrew/lib/python3.11/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_lg' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [22]:
import json

def provide_suggestions(user_review, aspects_with_suggestions, model):
    # Extract aspects using ABSA model
    aspects = model(user_review)

    # Print extracted aspects
    print("Extracted Aspects:")
    for aspect in aspects:
        aspect_text = aspect['span']
        polarity = aspect['polarity']
        print(f"Aspect: {aspect_text} (Polarity: {polarity})")

        # Match extracted aspects with suggestions in the JSON file and print suggestions
        found = False
        for entry in aspects_with_suggestions[polarity]:
            if isinstance(entry[0], str) and entry[0].lower() == aspect_text.lower():
                print(f"Suggestion for {aspect_text} (Polarity: {polarity}): {entry[1]}")
                found = True
                break

        # If no suggestions found for the aspect
        if not found:
            if polarity == 'negative':
                print(f"{aspect_text} needs improvement.")
            elif polarity == 'positive':
                print(f"{aspect_text} is loved by a certain group of audience.")
            elif polarity == 'neutral':
                print(f"There is a scope of improvement for {aspect_text}.")

In [26]:
user_review = input("Please enter your review: ")

# Call the function to provide suggestions
provide_suggestions(user_review, aspects_with_suggestions, model)

Extracted Aspects:
Aspect: gadget (Polarity: neutral)
Suggestion for gadget (Polarity: neutral): 4
Aspect: hearing aid device (Polarity: neutral)
There is a scope of improvement for hearing aid device.
Aspect: product (Polarity: neutral)
Suggestion for product (Polarity: neutral): 11
Aspect: price (Polarity: negative)
Suggestion for price (Polarity: negative): 10
Aspect: battery life (Polarity: positive)
Suggestion for battery life (Polarity: positive): 3
Aspect: battery solution (Polarity: positive)
battery solution is loved by a certain group of audience.
Aspect: price (Polarity: positive)
Suggestion for price (Polarity: positive): 9
